Want to make the Bubble chart a class. So a bubble chart "object" has:
(1) a dataset
(2) a variable for x-axis (and the ability to rename that variable something more descriptive)
(3) a variable for y-axis
(4) a variable for size of bubble
(5) a categorical variable for the color of the bubble (does it have to be categorical? I guess not absolutely)
(6) a variable on which to filter (in our example, year) (I think this might be a separate issue)

In [ ]:
#Make nkill = casualty & create a function that renames the columns something prettier & capitalizes it

In [ ]:
def rename_columns(data):
    data[]

In [ ]:
#Maybe we want to make a class that has the data and "data characteristics" like
#what's the name of the country column
#what's the name of the attacks column
#what's the name of the casualties column
#then the functions can be generalizeable to any dataset that has those features.

In [2]:
import pandas as pd
import numpy as np

gtd = pd.read_csv('C:\\Users\\Caroline\\Documents\\School\\Programming for Data Science\\Term Project\\whole_df.csv', low_memory=False, index_col=1).drop(['Unnamed: 0'], axis = 1)

bubble_chart = ['iyear', 'country_txt', 'region_txt', 'success', 'nkill']

gtd_bubble = gtd[bubble_chart]

In [3]:
bubble_chart = ['iyear', 'country_txt', 'region_txt', 'success', 'nkill']

gtd_bubble = gtd[bubble_chart]

In [52]:
gtd_bubble.describe()

C:\Users\Caroline\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,iyear,success,nkill
count,156772.000000,156772.000000,147827.000000
mean,2000.474083,0.903612,2.359237
std,12.982397,0.295124,11.421270
min,1970.000000,0.000000,0.000000
25%,1989.000000,1.000000,NaN
50%,2005.000000,1.000000,NaN
75%,2013.000000,1.000000,NaN
max,2015.000000,1.000000,1500.000000


In [4]:
#also seems generally useful, not specific to bubble chart
def group_years(data, years): #two methods that take 2 vars
    '''Turns years into groups of 5'''
    bins = np.arange(min(data) - years, max(data) + years, years)
    categories = pd.cut(data, bins)
    categories.name = 'Year Ranges'
    return categories

In [5]:
year_ranges = group_years(gtd_bubble['iyear'], 5)

def replace_years_with_range(data, years):
    '''Removes year column and merges range column'''
    year_ranges = group_years(data, years)
    data_replaced = pd.concat([gtd_bubble, pd.DataFrame(year_ranges)], axis = 1).drop(data.name, 1) #change this to be general
    return data_replaced

gtd_bubble.head()

,iyear,country_txt,region_txt,success,nkill
eventid,,,,,
197000000001,1970,Dominican Republic,Central America & Caribbean,1,1.0
197000000002,1970,Mexico,North America,1,0.0
197001000001,1970,Philippines,Southeast Asia,1,1.0
197001000002,1970,Greece,Western Europe,1,NaN
197001000003,1970,Japan,East Asia,1,NaN


In [6]:
gtd_bubble = replace_years_with_range(gtd_bubble['iyear'], 5)
gtd_bubble.head()

,country_txt,region_txt,success,nkill,Year Ranges
eventid,,,,,
197000000001,Dominican Republic,Central America & Caribbean,1,1.0,"(1965, 1970]"
197000000002,Mexico,North America,1,0.0,"(1965, 1970]"
197001000001,Philippines,Southeast Asia,1,1.0,"(1965, 1970]"
197001000002,Greece,Western Europe,1,NaN,"(1965, 1970]"
197001000003,Japan,East Asia,1,NaN,"(1965, 1970]"


In [ ]:
#Right now we group by the year range, but is there any reason we can't take the events that occurred in the year range
#and then group by bountry?

#I guess the reason I like it is then we deal with time all at once

#But the weird thing is this graph ought to be able to show any time range, and if it has to start over from the beginning
#I don't know how well that's going to work out

In [63]:
gtd_bubble.head()

,country_txt,region_txt,success,nkill,Year Ranges
eventid,,,,,
197000000001,Dominican Republic,Central America & Caribbean,1,1.0,"(1965, 1970]"
197000000002,Mexico,North America,1,0.0,"(1965, 1970]"
197001000001,Philippines,Southeast Asia,1,1.0,"(1965, 1970]"
197001000002,Greece,Western Europe,1,0.0,"(1965, 1970]"
197001000003,Japan,East Asia,1,0.0,"(1965, 1970]"


In [74]:
data = gtd_bubble
bubble_id = 'country_txt'
color = 'region_txt'
user_filter = 'Year Ranges'

data = data.loc[:,[bubble_id, color, user_filter]]
data['count'] = 1

type(pd.pivot_table(data, values='count', index=[bubble_id, color, user_filter], aggfunc=sum).dropna())

pandas.core.series.Series

In [27]:
class Bubble_Chart_Data():
    def __init__(self, data, bubble_id, color, user_filter):
        '''Defines attributes of the bubble chart'''
        self.data = data
        self.bubble_id = bubble_id
        self.color = color
        self.user_filter = user_filter
        self.agg_by_bubbleID = False
        self.bubble_size = 10 #initializes all the bubbles to the same size
    
    def count_by_bubbleID(self):
        '''Creates a count by bubble ID'''
        #this function has to be run first, need to reorder things to ensure this happens, do it in init function
        self.new_data = self.data.loc[:,[self.bubble_id, self.color, self.user_filter]]
        self.new_data['count'] = 1
        self.new_data = pd.pivot_table(self.new_data, values='count', index=[self.bubble_id, self.color, self.user_filter], aggfunc=sum).dropna()
        self.new_data = pd.DataFrame(self.new_data)
        self.new_data.reset_index(level=self.color, inplace=True)
        
    def sum_by_bubbleID(self, to_sum):
        '''Takes a column name and creates a sum '''
        #maybe allow you to specify which column, since we only want to do this for nkill
        index = [self.bubble_id, self.color, self.user_filter]
        summed = pd.pivot_table(self.data, values=to_sum, index=index, aggfunc=sum).dropna()
        summed = pd.DataFrame(summed)
        summed.reset_index(level=self.color, drop=True, inplace=True)
        self.new_data = summed.join(self.new_data, how='inner')
    
    def filter_for_user(self, user_input):
        '''Selects the part of the data that the user selected'''
        self.new_data = self.new_data.loc[(slice(None), user_input), :] 
    
    def set_bubble_size(self, bubble_size):
        '''bubble size must be a series with the bubble_id as the index'''
        #currently assume that you've already run filter_for_user, but I think we can solve this
        #by requiring the indexes to line up well
        self.new_data = pd.DataFrame(bubble_size).join(self.new_data, how='inner')
        self.bubble_size = bubble_size.columns[0] #currently assumes 1 column
        
    def add_color_dict(self):
        '''Creates a dictionary that maps the color variable to numbers'''
        #http://stackoverflow.com/questions/14885895/color-by-column-values-in-matplotlib
        color_cats = self.data[self.color].unique()
        colors = np.linspace(0, 1, len(color_cats))
        colordict = dict(zip(color_cats, colors))  
        self.new_data['color'] = self.new_data[self.color].apply(lambda x: colordict[x])
        
    #define a function that merges a bubble size and changes the self.bubble_size variable

In [28]:
bubble_chart = Bubble_Chart_Data(gtd_bubble, 'country_txt', 'region_txt', 'Year Ranges')

In [29]:
bubble_chart.count_by_bubbleID()

In [30]:
bubble_chart.sum_by_bubbleID('nkill')

In [31]:
bubble_chart.new_data.head()

nkill  region_txt  count
country_txt Year Ranges                           
Afghanistan (1970, 1975]    0.0  South Asia    1.0
            (1975, 1980]   53.0  South Asia    3.0
            (1985, 1990]  150.0  South Asia   24.0
            (1990, 1995]  144.0  South Asia   81.0
            (1995, 2000]  120.0  South Asia   29.0

In [32]:
bubble_chart.filter_for_user('(2010, 2015]')
bubble_chart.new_data.head()

,,nkill,region_txt,count
country_txt,Year Ranges,,,
Afghanistan,"(2010, 2015]",20359.0,South Asia,7077.0
Albania,"(2010, 2015]",1.0,Eastern Europe,7.0
Algeria,"(2010, 2015]",214.0,Middle East & North Africa,107.0
Argentina,"(2010, 2015]",0.0,South America,6.0
Armenia,"(2010, 2015]",0.0,Central Asia,3.0


In [33]:
def create_counts(data, group_by_column, column_to_count):
    '''Takes a column name and counts rows by that column'''
    data = data.loc[:,[group_by_column, column_to_count]].fillna(value = 1) #counts all, not just non-null
    grouped = data.groupby(group_by_column)
    return grouped.count()

In [34]:
total_attacks = create_counts(gtd_bubble, 'country_txt', 'success')
total_attacks.head()

,success
country_txt,
Afghanistan,9690
Albania,77
Algeria,2720
Andorra,1
Angola,491


In [35]:
bubble_chart.set_bubble_size(total_attacks)
bubble_chart.new_data.head()

,,success,nkill,region_txt,count
country_txt,Year Ranges,,,,
Afghanistan,"(2010, 2015]",9690,20359.0,South Asia,7077.0
Albania,"(2010, 2015]",77,1.0,Eastern Europe,7.0
Algeria,"(2010, 2015]",2720,214.0,Middle East & North Africa,107.0
Argentina,"(2010, 2015]",804,0.0,South America,6.0
Armenia,"(2010, 2015]",22,0.0,Central Asia,3.0


In [36]:
bubble_chart.add_color_dict()
bubble_chart.new_data.head()

,,success,nkill,region_txt,count,color
country_txt,Year Ranges,,,,,
Afghanistan,"(2010, 2015]",9690,20359.0,South Asia,7077.0,0.909091
Albania,"(2010, 2015]",77,1.0,Eastern Europe,7.0,0.545455
Algeria,"(2010, 2015]",2720,214.0,Middle East & North Africa,107.0,0.727273
Argentina,"(2010, 2015]",804,0.0,South America,6.0,0.454545
Armenia,"(2010, 2015]",22,0.0,Central Asia,3.0,1.000000


In [39]:
import bokeh

In [43]:
#got this snippet of code from: https://www.analyticsvidhya.com/blog/2015/05/data-visualization-python/

#need to add a legend & axis labels & format axis

from bokeh.plotting import show
from bokeh import mpl
#from bokeh.plotting import figure, show, output_file

import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.scatter(bubble_chart.new_data['count'], bubble_chart.new_data['nkill'],bubble_chart.new_data['success']/10, c = bubble_chart.new_data['color'], alpha = 0.5)
show(mpl.to_bokeh())

In [41]:
#http://bokeh.pydata.org/en/0.10.0/docs/gallery/scatter.html
#https://github.com/bokeh/bokeh/blob/master/examples/howto/interactive_bubble/gapminder.py
    
from numpy.random import random

from bokeh.plotting import figure, show, output_file

def mscatter(p, x, y, typestr):
    p.scatter(x, y, marker=typestr,
            line_color="#6666ee", fill_color="#ee6666", fill_alpha=0.5, size=12)

def mtext(p, x, y, textstr):
    p.text(x, y, text=[textstr],
         text_color="#449944", text_align="center", text_font_size="10pt")

output_file("markers.html")

p = figure(title="markers.py example")

N = 10

mscatter(p, random(N)+2, random(N)+1, "circle")
mscatter(p, random(N)+4, random(N)+1, "square")
mscatter(p, random(N)+6, random(N)+1, "triangle")
mscatter(p, random(N)+8, random(N)+1, "asterisk")

mscatter(p, random(N)+2, random(N)+4, "circle_x")
mscatter(p, random(N)+4, random(N)+4, "square_x")
mscatter(p, random(N)+6, random(N)+4, "inverted_triangle")
mscatter(p, random(N)+8, random(N)+4, "x")

mscatter(p, random(N)+2, random(N)+7, "circle_cross")
mscatter(p, random(N)+4, random(N)+7, "square_cross")
mscatter(p, random(N)+6, random(N)+7, "diamond")
mscatter(p, random(N)+8, random(N)+7, "cross")

mtext(p, [2.5], [0.5], "circle / o")
mtext(p, [4.5], [0.5], "square")
mtext(p, [6.5], [0.5], "triangle")
mtext(p, [8.5], [0.5], "asterisk / *")

mtext(p, [2.5], [3.5], "circle_x / ox")
mtext(p, [4.5], [3.5], "square_x")
mtext(p, [6.5], [3.5], "inverted_triangle")
mtext(p, [8.5], [3.5], "x")

mtext(p, [2.5], [6.5], "circle_cross / o+")
mtext(p, [4.5], [6.5], "square_cross")
mtext(p, [6.5], [6.5], "diamond")
mtext(p, [8.5], [6.5], "cross / +")

show(p)  # open a browser

Also considered doing a bubble chart where the x axis is percentage change in # killed, y axis is percentage change in # attacks, and size of bubble is # incidents

Next step might be scroll-over labels, b/c I think that would be really helpful

In [101]:
#this is useful in general, don't make it specific to bubble chart
def create_total(to_sum, sum_by):
    '''takes two series of the same length, sums one by the other, returns a series with the result'''
    data = pd.concat([sum_by, to_sum], axis = 1)
    totaled = pd.pivot_table(data, index = sum_by.name, aggfunc=sum)
    totaled.columns = ['total ' + to_sum.name]
    return pd.Series(totaled['total ' + to_sum.name])